In [1]:
! pip install clickhouse_connect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.2/966.2 kB 13.1 MB/s eta 0:00:0000:0100:01


In [2]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='91.224.86.248', 
                                       port=8123,
                                       username='itutitam',
                                       password='g1UrkTy7oQFWfJD2s')

# Найдем средние вектора для тематических кластеров

In [5]:
TABLENAME = 'embeddings'
data = client.query_df(f'SELECT id, clip_emb FROM {TABLENAME}')

In [6]:
data = data.drop_duplicates(subset='id')

In [10]:
import pandas as pd

clusters = pd.read_csv('/kaggle/input/clusters-yappy/final_clusters.csv').drop(columns='Unnamed: 0')

In [31]:
clusters_ids = [list(map(int, clusters.id[i][1:-1].split(', '))) for i in range(len(clusters))]

In [68]:
embeddings = []

for ids in clusters_ids:
    total_emb = np.array([0 for i in range(512)], dtype='float64')
    total_count = 0
    for id_ in ids:
        total_emb += np.array(data['clip_emb'][data['id'] == id_].tolist()[0])
        total_count += 1
    embeddings.append(total_emb / total_count)

In [70]:
clusters['mean_emb'] = embeddings

In [71]:
clusters

,name,cluster,id,links,mean_emb
0,Brawl Stars,72,"[36736, 20354, 34818, 20998, 28039, 27784, 321...",['https://cdn-st.rutubelist.ru/media/88/e4/0f1...,"[-0.005409819172373751, 0.02004028727450142, 0..."
1,DIY & Art,65,"[4609, 12292, 30214, 28168, 14344, 12520, 2254...",['https://cdn-st.rutubelist.ru/media/37/5a/669...,"[-0.013120555008171615, -0.0012014100940254816..."
2,Dota 2,69,"[22533, 30215, 18952, 28171, 28688, 5649, 8210...",['https://cdn-st.rutubelist.ru/media/ab/9c/e75...,"[-0.006984333054056948, 0.018158942180176833, ..."
3,FIFA,14,"[7936, 6918, 25992, 23050, 31501, 36365, 25746...",['https://cdn-st.rutubelist.ru/media/51/df/540...,"[0.04348502891997878, -0.024786487977717816, 0..."
4,Genshin Impact,93,"[15872, 19144, 31627, 21711, 25488, 10223, 500...",['https://cdn-st.rutubelist.ru/media/49/1d/de4...,"[-0.00118827230055883, 0.0363957371030535, -0...."
5,Автомобили,83,"[2048, 22021, 3080, 19978, 26122, 32267, 2573,...",['https://cdn-st.rutubelist.ru/media/47/27/b97...,"[0.02835784691451649, 0.01718079283118601, -0...."
6,Аниме,92,"[16397, 30734, 2062, 15, 22545, 4115, 35, 2256...",['https://cdn-st.rutubelist.ru/media/98/a6/96a...,"[-0.010619504250082847, 0.02774934457885521, 0..."
7,Аутфиты,47,"[18437, 22535, 14345, 10292, 4149, 6203, 12348...",['https://cdn-st.rutubelist.ru/media/67/d1/1d9...,"[-0.00047661785384111807, 0.02638165764814939,..."
8,Баскетбол,28,"[18820, 22149, 9227, 530, 3475, 36370, 17298, ...",['https://cdn-st.rutubelist.ru/media/ed/c7/abf...,"[0.03348925502293489, -0.0052814247732220045, ..."
9,Гитара,3,"[33416, 7436, 25978, 1682, 26516, 30359, 7704,...",['https://cdn-st.rutubelist.ru/media/f3/03/7bd...,"[0.031190507840409946, 0.011050433878083892, 0..."


In [72]:
clusters.to_csv('final_clusters.csv')

In [98]:
from sklearn.metrics.pairwise import cosine_similarity

# пример поиска подходящего кластера для поступившего видео

clip_emb = np.array([0.5 for i in range(512)], dtype='float64') # CLIP-эмбеддинг поступившего видео
clusters['sim'] = clusters['mean_emb'].apply(lambda emb: cosine_similarity(emb.reshape(1, -1), clip_emb.reshape(1, -1))[0][0])
clusters = clusters.sort_values(by='sim', ascending=False)
class_ = clusters.head(1).name.tolist()[0]

'Пародии'

# Найдем средние вектора для кластеров, основанных на лицах

In [101]:
df_faces = client.query_df(f'SELECT id, face_emb FROM faces_embeddings')

In [114]:
df_faces

,id,face_emb
0,2411,"[-0.04813425801694393, 0.10891282930970192, 0...."
1,2412,"[-0.09570123255252838, 0.03189397603273392, 0...."
2,2413,"[-0.18776406347751617, 0.019975094124674797, 0..."
3,2414,"[-0.11057936400175095, -0.022639945149421692, ..."
4,2416,"[-0.10623737704008818, 0.035882167401723564, 0..."
...,...,...
30891,45576,"[-0.15068353712558746, 0.11511006951332092, 0...."
30892,45576,"[-0.0925418883562088, 0.14501436054706573, 0.0..."
30893,45576,"[-0.05188953876495361, 0.0075677186250686646, ..."
30894,45579,"[-0.09036068618297577, 0.14766216278076172, 0...."


In [106]:
face_clusters = pd.read_csv('/kaggle/input/face-clusters-yappy/faces-31000-videos-filtered.csv').drop(columns='Unnamed: 0')

In [109]:
clusters_ids = [list(map(int, face_clusters.id[i][1:-1].split(', '))) for i in range(len(face_clusters))]

In [123]:
embeddings = []

for ids in clusters_ids:
    total_emb = np.array([0 for i in range(128)], dtype='float64')
    total_count = 0
    for id_ in ids:
        total_emb += np.array(df_faces['face_emb'][df_faces['id'] == id_].tolist()[0])
        total_count += 1
    embeddings.append(total_emb / total_count)

In [124]:
face_clusters['mean_emb'] = embeddings

In [126]:
face_clusters

,cluster,id,links,name,mean_emb
0,2,"[42372, 2678, 5185, 7520, 18003, 18070, 27340,...",['https://cdn-st.rutubelist.ru/media/91/f5/1f2...,про фильмы,"[-0.06411567758768796, 0.08846340291202068, 0...."
1,3,"[1478, 2029, 2673, 2948, 3811, 3999, 7777, 821...",['https://cdn-st.rutubelist.ru/media/4b/3e/0d6...,про игры,"[-0.05034815381152619, 0.07844406987765226, -0..."
2,4,"[39666, 2301, 5800, 6051, 6106, 12246, 13507, ...",['https://cdn-st.rutubelist.ru/media/20/1a/93e...,про телефоны,"[-0.1266581850902488, 0.09319817132006089, 0.0..."
3,5,"[4603, 9313, 14785, 16589, 26235, 26305, 27428...",['https://cdn-st.rutubelist.ru/media/3f/c9/080...,интересные факты,"[-0.07446376384737399, 0.09270460776483211, -0..."
4,6,"[42643, 298, 535, 2220, 6774, 7864, 8887, 9145...",['https://cdn-st.rutubelist.ru/media/9d/e7/236...,про уход за кожей,"[-0.07857688225894457, 0.08131374490247774, 0...."
...,...,...,...,...,...
75,122,"[45609, 1485, 2025, 2912, 3569, 4623, 4772, 52...",['https://cdn-st.rutubelist.ru/media/0e/5a/0b3...,про макияж,"[-0.04830516380367472, -0.010115716442027512, ..."
76,123,"[42394, 45985, 181, 1311, 3095, 3230, 4497, 81...",['https://cdn-st.rutubelist.ru/media/e3/cb/66b...,про русский язык,"[-0.13700648444785696, 0.13915244427145176, 0...."
77,124,"[42545, 45881, 45891, 345, 1307, 1456, 3657, 5...",['https://cdn-st.rutubelist.ru/media/66/cc/c1b...,про футбол,"[-0.1019461531248655, 0.03949003741517016, 0.0..."
78,127,"[42506, 690, 1419, 3186, 3843, 5272, 5554, 594...",['https://cdn-st.rutubelist.ru/media/df/ec/efa...,про восточные страны,"[-0.04343558698328155, 0.0037772401198956436, ..."


In [127]:
face_clusters.to_csv('faces-31000-videos-filtered.csv')

In [132]:
from sklearn.metrics.pairwise import cosine_similarity

# пример поиска подходящего кластера для поступившего видео

face_emb = np.array([0.5 for i in range(128)], dtype='float64') # эмбеддинг лица поступившего видео
face_clusters['sim'] = face_clusters['mean_emb'].apply(lambda emb: cosine_similarity(emb.reshape(1, -1), face_emb.reshape(1, -1))[0][0])
face_clusters = face_clusters.sort_values(by='sim', ascending=False)
class_ = face_clusters.head(1).cluster.tolist()[0]